<a href="https://colab.research.google.com/github/Lee-Sammy/Hyunseo_DTSC3020_Fall2025/blob/main/HyunseoLee_Assignment_6_WebScraping_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 6 (4 points) — Web Scraping

In this assignment you will complete **two questions**. The **deadline is posted on Canvas**.


## Assignment Guide (Read Me First)

- This notebook provides an **Install Required Libraries** cell and a **Common Imports & Polite Headers** cell. Run them first.
- Each question includes a **skeleton**. The skeleton is **not** a solution; it is a lightweight scaffold you may reuse.
- Under each skeleton you will find a **“Write your answer here”** code cell. Implement your scraping, cleaning, and saving logic there.
- When your code is complete, run the **Runner** cell to print a Top‑15 preview and save the CSV.
- Expected outputs:
  - **Q1:** `data_q1.csv` + Top‑15 sorted by the specified numeric column.
  - **Q2:** `data_q2.csv` + Top‑15 sorted by `points`.


In [8]:
1) #Install Required Libraries
!pip -q install requests beautifulsoup4 lxml pandas
print("Dependencies installed.")


SyntaxError: unmatched ')' (ipython-input-565355372.py, line 1)

### 2) Common Imports & Polite Headers

In [2]:
# Common Imports & Polite Headers
import re, sys, pandas as pd, requests
from bs4 import BeautifulSoup
HEADERS = {"User-Agent": (
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
    "(KHTML, like Gecko) Chrome/122.0 Safari/537.36")}
def fetch_html(url: str, timeout: int = 20) -> str:
    r = requests.get(url, headers=HEADERS, timeout=timeout)
    r.raise_for_status()
    return r.text
def flatten_headers(df: pd.DataFrame) -> pd.DataFrame:
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [" ".join([str(x) for x in tup if str(x)!="nan"]).strip()
                      for tup in df.columns.values]
    else:
        df.columns = [str(c).strip() for c in df.columns]
    return df
print("Common helpers loaded.")


Common helpers loaded.


## Question 1 — IBAN Country Codes (table)
**URL:** https://www.iban.com/country-codes  
**Extract at least:** `Country`, `Alpha-2`, `Alpha-3`, `Numeric` (≥4 cols; you may add more)  
**Clean:** trim spaces; `Alpha-2/Alpha-3` → **UPPERCASE**; `Numeric` → **int** (nullable OK)  
**Output:** write **`data_q1.csv`** and **print a Top-15** sorted by `Numeric` (desc, no charts)  
**Deliverables:** notebook + `data_q1.csv` + short `README.md` (URL, steps, 1 limitation)

**Tip:** You can use `pandas.read_html(html)` to read tables and then pick one with ≥3 columns.


In [3]:
# --- Q1 Skeleton (fill the TODOs) ---
def q1_read_table(html: str) -> pd.DataFrame:
    """Return the first table with >= 3 columns from the HTML.
    TODO: implement with pd.read_html(html), pick a reasonable table, then flatten headers.
    """
    raise NotImplementedError("TODO: implement q1_read_table")

def q1_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean columns: strip, UPPER Alpha-2/Alpha-3, cast Numeric to int (nullable), drop invalids.
    TODO: implement cleaning steps.
    """
    raise NotImplementedError("TODO: implement q1_clean")

def q1_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort descending by Numeric and return Top-N.
    TODO: implement.
    """
    raise NotImplementedError("TODO: implement q1_sort_top")


In [4]:
# Q1 — Write your answer here

def q1_read_table(html: str) -> pd.DataFrame:
    """
    Return the first table with >= 3 columns from the HTML.
    Uses pandas.read_html and flattens headers.
    """
    tables = pd.read_html(html)
    if not tables:
        raise ValueError("No tables found in HTML.")

    df = None
    for t in tables:
        if t.shape[1] >= 3:
            df = t.copy()
            break

    if df is None:
        raise ValueError("No table with at least 3 columns found.")

    df = flatten_headers(df)
    return df


def q1_clean(df: pd.DataFrame) -> pd.DataFrame:
    """
    Clean columns: strip whitespace, UPPER Alpha-2/Alpha-3,
    cast Numeric to nullable int, drop invalid rows.
    """
    df = df.copy()
    df = flatten_headers(df)

    col_map = {}
    for c in df.columns:
        cl = c.strip().lower()
        if "country" in cl:
            col_map[c] = "Country"
        elif "alpha-2" in cl:
            col_map[c] = "Alpha-2"
        elif "alpha-3" in cl:
            col_map[c] = "Alpha-3"
        elif "numeric" in cl:
            col_map[c] = "Numeric"

    df = df.rename(columns=col_map)

    wanted = ["Country", "Alpha-2", "Alpha-3", "Numeric"]
    existing = [c for c in wanted if c in df.columns]
    df = df[existing]

    for col in ["Country", "Alpha-2", "Alpha-3"]:
        if col in df.columns:
            df[col] = df[col].astype(str).str.strip()

    for col in ["Alpha-2", "Alpha-3"]:
        if col in df.columns:
            df[col] = df[col].str.upper()

    if "Numeric" in df.columns:
        df["Numeric"] = pd.to_numeric(df["Numeric"], errors="coerce").astype("Int64")

    if "Country" in df.columns:
        df = df[df["Country"].str.strip() != ""]
    if "Alpha-2" in df.columns:
        df = df[df["Alpha-2"].str.strip() != ""]

    df = df.reset_index(drop=True)
    return df


def q1_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """
    Sort descending by Numeric and return Top-N.
    """
    if "Numeric" in df.columns:
        return df.sort_values("Numeric", ascending=False).head(top)
    else:
        return df.head(top)





In [5]:
url = "https://www.iban.com/country-codes"

html = fetch_html(url)
df_raw = q1_read_table(html)
df_clean = q1_clean(df_raw)

# Top-15 sorted by Numeric desc
df_top15 = q1_sort_top(df_clean, top=15)
print(df_top15)

# Save full cleaned data
df_clean.to_csv("data_q1.csv", index=False)
print("\nSaved data_q1.csv with", len(df_clean), "rows.")

                                               Country Alpha-2 Alpha-3  \
247                                             Zambia      ZM     ZMB   
246                                              Yemen      YE     YEM   
192                                              Samoa      WS     WSM   
244                                  Wallis and Futuna      WF     WLF   
240                 Venezuela (Bolivarian Republic of)      VE     VEN   
238                                         Uzbekistan      UZ     UZB   
237                                            Uruguay      UY     URY   
35                                        Burkina Faso      BF     BFA   
243                              Virgin Islands (U.S.)      VI     VIR   
236                     United States of America (the)      US     USA   
219                       Tanzania, United Republic of      TZ     TZA   
108                                        Isle of Man      IM     IMN   
113                                   

/tmp/ipython-input-2698840710.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html)


## Question 2 — Hacker News (front page)
**URL:** https://news.ycombinator.com/  
**Extract at least:** `rank`, `title`, `link`, `points`, `comments` (user optional)  
**Clean:** cast `points`/`comments`/`rank` → **int** (non-digits → 0), fill missing text fields  
**Output:** write **`data_q2.csv`** and **print a Top-15** sorted by `points` (desc, no charts)  
**Tip:** Each story is a `.athing` row; details (points/comments/user) are in the next `<tr>` with `.subtext`.


In [ ]:
# --- Q2 Skeleton (fill the TODOs) ---
def q2_parse_items(html: str) -> pd.DataFrame:
    """Parse front page items into DataFrame columns:
       rank, title, link, points, comments, user (optional).
    TODO: implement with BeautifulSoup on '.athing' and its sibling '.subtext'.
    """
    raise NotImplementedError("TODO: implement q2_parse_items")

def q2_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean numeric fields and fill missing values.
    TODO: cast points/comments/rank to int (non-digits -> 0). Fill text fields.
    """
    raise NotImplementedError("TODO: implement q2_clean")

def q2_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort by points desc and return Top-N. TODO: implement."""
    raise NotImplementedError("TODO: implement q2_sort_top")


In [6]:
# Q2 — Write your answer here
def q2_parse_items(html: str) -> pd.DataFrame:
    """Parse front page items into DataFrame columns:
       rank, title, link, points, comments, user (optional).
    """
    soup = BeautifulSoup(html, "lxml")
    rows = []

    for item in soup.select("tr.athing"):
        rank_el = item.select_one(".rank")
        rank_text = rank_el.get_text(strip=True) if rank_el else ""
        rank_digits = re.findall(r"\d+", rank_text)
        rank = int(rank_digits[0]) if rank_digits else 0

        title_link = None
        title_el = item.select_one(".titleline a")
        if title_el:
            title = title_el.get_text(strip=True)
            link = title_el.get("href", "").strip()
        else:
            title_el = item.select_one("a.storylink")
            title = title_el.get_text(strip=True) if title_el else ""
            link = title_el.get("href", "").strip() if title_el else ""

        sub = item.find_next_sibling("tr")
        points = 0
        comments = 0
        user = ""

        if sub:
            subtext = sub.select_one(".subtext")
            if subtext:
                score_el = subtext.select_one(".score")
                if score_el:
                    score_text = score_el.get_text(strip=True)  # "123 points"
                    score_digits = re.findall(r"\d+", score_text)
                    points = int(score_digits[0]) if score_digits else 0

                user_el = subtext.select_one("a.hnuser")
                if user_el:
                    user = user_el.get_text(strip=True)

                a_tags = subtext.select("a")
                if a_tags:
                    last_a = a_tags[-1]
                    c_text = last_a.get_text(strip=True)
                    c_digits = re.findall(r"\d+", c_text)
                    comments = int(c_digits[0]) if c_digits else 0

        rows.append(
            {
                "rank": rank,
                "title": title,
                "link": link,
                "points": points,
                "comments": comments,
                "user": user,
            }
        )

    df = pd.DataFrame(rows)
    return df


def q2_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean numeric fields and fill missing values."""
    df = df.copy()

    for col in ["rank", "points", "comments"]:
        df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0).astype(int)

    for col in ["title", "link", "user"]:
        if col in df.columns:
            df[col] = df[col].fillna("").astype(str).str.strip()

    return df


def q2_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort by points desc and return Top-N."""
    if "points" in df.columns:
        return df.sort_values("points", ascending=False).head(top)
    else:
        return df.head(top)




In [7]:
url = "https://news.ycombinator.com/"

html = fetch_html(url)
df_raw_q2 = q2_parse_items(html)
df_clean_q2 = q2_clean(df_raw_q2)

df_top15_q2 = q2_sort_top(df_clean_q2, top=15)
print(df_top15_q2)

df_clean_q2.to_csv("data_q2.csv", index=False)
print("\nSaved data_q2.csv with", len(df_clean_q2), "rows.")


    rank                                              title  \
14    15            FBI tries to unmask owner of archive.is   
4      5  Kimi K2 Thinking, a SOTA open-source trillion-...   
22    23             ICC ditches Microsoft 365 for openDesk   
0      1                          You should write an agent   
10    11  Open Source Implementation of Apple's Private ...   
5      6           Two billion email addresses were exposed   
25    26  I may have found a way to spot U.S. at-sea str...   
6      7  Show HN: I scraped 3B Goodreads reviews to tra...   
20    21    Mathematical exploration and discovery at scale   
9     10                           Swift on FreeBSD Preview   
15    16                            Eating stinging nettles   
3      4                              Game design is simple   
17    18  I analyzed the lineups at the most popular nig...   
28    29  Show HN: qqqa – A fast, stateless LLM-powered ...   
11    12             LLMs encode how difficult problems